**Intermediate Assignment 1: Importing and Joining Data**

*Part 1: Food Environment Atlas*

In [69]:
# Import packages
import pandas as pd
import requests

In [70]:
# Import Food Environment Atlas, get list of sheets to create dataset
path = r'C:\Users\JohnTaylor\DSTP_Assignments\Assignment1\Data\Food Environment Atlas - Copy.xlsx'
excel = pd.ExcelFile(path)
print(excel.sheet_names)
targets = ['ACCESS', 'STORES', 'RESTAURANTS', 'ASSISTANCE', 'INSECURITY', 'TAXES', 'LOCAL', 'HEALTH', 'SOCIOECONOMIC']

['Read_Me', ' Variable List', 'Supplemental Data-County', 'Supplemental Data - State', 'ACCESS', 'STORES', 'RESTAURANTS', 'ASSISTANCE', 'INSECURITY', 'TAXES', 'LOCAL', 'HEALTH', 'SOCIOECONOMIC']


Prepping FIPS Codes for Merge

In [71]:
# Convert sheets to dataframes and check FIPS columns
df_dict = {}
print('Checking columns...')
for sheet in targets:
    df_dict[sheet] = excel.parse(sheet)
    # check if there is a 'FIPS' column and the data type
    if 'FIPS' in df_dict[sheet].columns:
        print('{} has a "FIPS" column with {} rows and the data type is {}'.format(sheet, len(df_dict[sheet]), df_dict[sheet]['FIPS'].dtypes))
    else:
        print('{} does not have a "FIPS" column'.format(sheet))

# check for NA values in FIPS columns
print('Checking values...')
for sheet in df_dict:
    check = df_dict[sheet]['FIPS'].isnull().any()
    if check:
        print('{} contains 1 or more null values in the FIPS column'.format(sheet))
        print(df_dict["LOCAL"]['FIPS'].isnull())


Checking columns...
ACCESS has a "FIPS" column with 3143 rows and the data type is int64
STORES has a "FIPS" column with 3143 rows and the data type is int64
RESTAURANTS has a "FIPS" column with 3143 rows and the data type is int64
ASSISTANCE has a "FIPS" column with 3143 rows and the data type is int64
INSECURITY has a "FIPS" column with 3143 rows and the data type is int64
TAXES has a "FIPS" column with 3143 rows and the data type is int64
LOCAL has a "FIPS" column with 3144 rows and the data type is float64
HEALTH has a "FIPS" column with 3143 rows and the data type is int64
SOCIOECONOMIC has a "FIPS" column with 3143 rows and the data type is int64
Checking values...
LOCAL contains 1 or more null values in the FIPS column
0       False
1       False
2       False
3       False
4       False
        ...  
3139    False
3140    False
3141    False
3142    False
3143     True
Name: FIPS, Length: 3144, dtype: bool


In [72]:
# Read row 3144 and surroundings of LOCAL dataframe
df_dict['LOCAL'].iloc[-15:, :]

,FIPS,State,County,DIRSALES_FARMS07,DIRSALES_FARMS12,PCH_DIRSALES_FARMS_07_12,PCT_LOCLFARM07,PCT_LOCLFARM12,PCT_LOCLSALE07,PCT_LOCLSALE12,DIRSALES07,DIRSALES12,PCH_DIRSALES_07_12,PC_DIRSALES07,PC_DIRSALES12,PCH_PC_DIRSALES_07_12,FMRKT13,FMRKT18,PCH_FMRKT_13_18,FMRKTPTH13,FMRKTPTH18,PCH_FMRKTPTH_13_18,FMRKT_SNAP18,PCT_FMRKT_SNAP18,FMRKT_WIC18,PCT_FMRKT_WIC18,FMRKT_WICCASH18,PCT_FMRKT_WICCASH18,FMRKT_SFMNP18,PCT_FMRKT_SFMNP18,FMRKT_CREDIT18,PCT_FMRKT_CREDIT18,FMRKT_FRVEG18,PCT_FMRKT_FRVEG18,FMRKT_ANMLPROD18,PCT_FMRKT_ANMLPROD18,FMRKT_BAKED18,PCT_FMRKT_BAKED18,FMRKT_OTHERFOOD18,PCT_FMRKT_OTHERFOOD18,...,PCH_ORCHARD_FARMS_07_12,ORCHARD_ACRES07,ORCHARD_ACRES12,PCH_ORCHARD_ACRES_07_12,ORCHARD_ACRESPTH07,ORCHARD_ACRESPTH12,PCH_ORCHARD_ACRESPTH_07_12,BERRY_FARMS07,BERRY_FARMS12,PCH_BERRY_FARMS_07_12,BERRY_ACRES07,BERRY_ACRES12,PCH_BERRY_ACRES_07_12,BERRY_ACRESPTH07,BERRY_ACRESPTH12,PCH_BERRY_ACRESPTH_07_12,SLHOUSE07,SLHOUSE12,PCH_SLHOUSE_07_12,GHVEG_FARMS07,GHVEG_FARMS12,PCH_GHVEG_FARMS_07_12,GHVEG_SQFT07,GHVEG_SQFT12,PCH_GHVEG_SQFT_07_12,GHVEG_SQFTPTH07,GHVEG_SQFTPTH12,PCH_GHVEG_SQFTPTH_07_12,FOODHUB18,CSA07,CSA12,PCH_CSA_07_12,AGRITRSM_OPS07,AGRITRSM_OPS12,PCH_AGRITRSM_OPS_07_12,AGRITRSM_RCT07,AGRITRSM_RCT12,PCH_AGRITRSM_RCT_07_12,FARM_TO_SCHOOL13,FARM_TO_SCHOOL15
3129,56019.0,WY,Johnson,15.0,28.0,86.666667,4.702194,7.821229,0.175081,0.191452,49.0,99.0,102.040816,6.015222,11.498258,91.152670,1.0,1.0,0.0,0.115727,0.118203,2.139485,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,100.000000,1.0,100.0,1.0,100.000000,1.0,100.0,1.0,100.000000,...,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,1.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.000000,0.000000,NaN,NaN,0.0,1.0,NaN,29.0,38.0,31.034483,747000.0,1760000.0,135.609103,0.000000,0.0
3130,56021.0,WY,Laramie,42.0,58.0,38.095238,4.976303,5.197133,0.112012,0.104328,139.0,199.0,43.165468,1.585780,2.097077,32.242564,4.0,3.0,-25.0,0.041795,0.030310,-27.477873,1.0,33.333332,0.0,0.0,0.0,0.0,0.0,0.000000,3.0,100.000000,3.0,100.0,3.0,100.000000,3.0,100.0,3.0,100.000000,...,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,1.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,2.0,2.0,0.0,0.0,6.0,NaN,0.0,8400.0,NaN,0.000000,88.519822,NaN,NaN,5.0,4.0,-20.000000,16.0,22.0,37.500000,173000.0,378000.0,118.497110,1.000000,0.0
3131,56023.0,WY,Lincoln,36.0,46.0,27.777778,6.728972,7.565789,0.691834,0.541282,208.0,221.0,6.250000,12.225945,12.316781,0.742978,1.0,2.0,100.0,0.054597,0.102912,88.494392,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,50.000000,1.0,50.0,1.0,50.000000,1.0,50.0,1.0,50.000000,...,NaN,0.0,0.0,NaN,0.000000,0.000000,NaN,0.0,1.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,3.0,3.0,0.0,15800.0,38024.0,140.658228,928.701581,2119.155102,128.18472,NaN,11.0,6.0,-45.454545,13.0,9.0,-30.769231,NaN,NaN,NaN,1.000000,0.0
3132,56025.0,WY,Natrona,32.0,20.0,-37.500000,7.748184,5.037783,0.406678,0.321506,133.0,138.0,3.759398,1.837905,1.753517,-4.591559,2.0,3.0,50.0,0.024661,0.037919,53.765408,2.0,66.666664,0.0,0.0,0.0,0.0,0.0,0.000000,2.0,66.666664,3.0,100.0,3.0,100.000000,3.0,100.0,3.0,100.000000,...,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,1.0,1.0,0.0,0.0,1.0,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,NaN,3.0,0.0,-100.000000,19.0,17.0,-10.526316,131000.0,119000.0,-9.160305,0.000000,0.0
3133,56027.0,WY,Niobrara,4.0,9.0,125.000000,1.702128,3.846154,0.118736,0.088257,44.0,40.0,-9.090909,18.280017,16.161616,-11.588613,1.0,1.0,0.0,0.392927,0.418760,6.574539,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,NaN,0.0,0.0,NaN,0.000000,0.000000,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.000000,0.000000,NaN,NaN,0.0,0.0,NaN,18.0,6.0,-66.666667,35000.0,7000.0,-80.000000,0.000000,0.0
3134,56029.0,WY,Park,41.0,52.0,26.829268,5.242967,6.046512,0.635891,0.218378,520.0,219.0,-57.884615,19.111323,7.587569,-60.298046,1.0,4.0,300.0,0.034427,0.136407,296.221497,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,3.0,75.000000,3.0,75.0,2.0,50.000000,2.0,50.0,

In [73]:
# Remove null row from LOCAL
df_dict['LOCAL'] = df_dict['LOCAL'].iloc[0:-1, :].copy() 
df_dict['LOCAL'].iloc[-15:, :]
print(df_dict['LOCAL']['FIPS'].isnull().any())

# Convert LOCAL['FIPS'] to int
df_dict['LOCAL']['FIPS'] = df_dict['LOCAL']['FIPS'].astype('int')
df_dict['LOCAL']['FIPS'].dtypes


False


dtype('int32')

In [74]:
# Check if FIPS columns match between dataframes
example_df = df_dict['ACCESS']
fips_list = list(example_df['FIPS'].unique())

for sheet in df_dict:
    print('{} : {}'.format(sheet, fips_list==list(df_dict[sheet]['FIPS'].unique())))


ACCESS : True
STORES : True
RESTAURANTS : True
ASSISTANCE : True
INSECURITY : True
TAXES : True
LOCAL : True
HEALTH : True
SOCIOECONOMIC : True


Merge dataframes together into EnvironmentAll

In [75]:
EnvironmentAll = df_dict[targets[0]]

# Add additional sheets
for sheet in targets[1:]:
    df_to_merge = df_dict[sheet].drop(columns=['State', 'County'])
    EnvironmentAll = EnvironmentAll.merge(df_to_merge, on='FIPS')

EnvironmentAll.head()
    

,FIPS,State,County,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,PCH_LACCESS_LOWI_10_15,PCT_LACCESS_LOWI10,PCT_LACCESS_LOWI15,LACCESS_HHNV10,LACCESS_HHNV15,PCH_LACCESS_HHNV_10_15,PCT_LACCESS_HHNV10,PCT_LACCESS_HHNV15,LACCESS_SNAP15,PCT_LACCESS_SNAP15,LACCESS_CHILD10,LACCESS_CHILD15,LACCESS_CHILD_10_15,PCT_LACCESS_CHILD10,PCT_LACCESS_CHILD15,LACCESS_SENIORS10,LACCESS_SENIORS15,PCH_LACCESS_SENIORS_10_15,PCT_LACCESS_SENIORS10,PCT_LACCESS_SENIORS15,LACCESS_WHITE15,PCT_LACCESS_WHITE15,LACCESS_BLACK15,PCT_LACCESS_BLACK15,LACCESS_HISP15,PCT_LACCESS_HISP15,LACCESS_NHASIAN15,PCT_LACCESS_NHASIAN15,LACCESS_NHNA15,PCT_LACCESS_NHNA15,...,GHVEG_SQFTPTH12,PCH_GHVEG_SQFTPTH_07_12,FOODHUB18,CSA07,CSA12,PCH_CSA_07_12,AGRITRSM_OPS07,AGRITRSM_OPS12,PCH_AGRITRSM_OPS_07_12,AGRITRSM_RCT07,AGRITRSM_RCT12,PCH_AGRITRSM_RCT_07_12,FARM_TO_SCHOOL13,FARM_TO_SCHOOL15,PCT_DIABETES_ADULTS08,PCT_DIABETES_ADULTS13,PCT_OBESE_ADULTS12,PCT_OBESE_ADULTS17,PCT_HSPA17,RECFAC11,RECFAC16,PCH_RECFAC_11_16,RECFACPTH11,RECFACPTH16,PCH_RECFACPTH_11_16,PCT_NHWHITE10,PCT_NHBLACK10,PCT_HISP10,PCT_NHASIAN10,PCT_NHNA10,PCT_NHPI10,PCT_65OLDER10,PCT_18YOUNGER10,MEDHHINC15,POVRATE15,PERPOV10,CHILDPOVRATE15,PERCHLDPOV10,METRO13,POPLOSS10
0,1001,AL,Autauga,18428.439685,17496.693038,-5.056026,33.769657,32.062255,5344.427472,6543.676824,22.439248,9.793530,11.991125,664.216487,677.672769,2.025888,3.284786,3.351332,931.935186,4.608749,4822.500269,4616.971480,-4.261872,8.837112,8.460485,2388.233228,2180.809285,-8.685247,4.376378,3.996279,12640.615414,23.163613,4216.473194,7.726582,471.136164,0.863345,86.767975,0.159000,61.169869,0.112092,...,0.000000,NaN,NaN,2.0,3.0,50.000000,7.0,10.0,42.857143,228000.0,146000.0,-35.964912,NaN,0.0,11.4,13.0,33.0,36.3,NaN,4,6,50.000000,0.072465,0.108542,49.785629,77.246156,17.582599,2.400542,0.855766,0.397647,0.040314,11.995382,26.777959,56580.0,12.7,0,18.8,0,1,0.0
1,1003,AL,Baldwin,35210.814078,30561.264430,-13.204891,19.318473,16.767489,9952.144027,9886.831137,-0.656270,5.460261,5.424427,1571.779915,1394.162766,-11.300383,2.147827,1.905114,950.535290,1.298900,7916.131932,7007.972675,-11.472260,4.343199,3.844936,6404.008603,5580.662790,-12.856726,3.513570,3.061840,25483.186811,13.981393,3540.965826,1.942757,1377.874834,0.755973,212.946378,0.116833,181.649648,0.099662,...,326.477447,NaN,NaN,13.0,7.0,-46.153846,18.0,16.0,-11.111111,124000.0,204000.0,64.516129,0.0,1.0,9.8,10.4,33.0,36.3,NaN,16,21,31.250000,0.085775,0.101200,17.983256,83.504787,9.308425,4.384824,0.735193,0.628755,0.043343,16.771185,22.987408,52387.0,12.9,0,19.6,0,1,0.0
2,1005,AL,Barbour,5722.305602,6069.523628,6.067799,20.840972,22.105560,3135.676086,2948.790251,-5.959985,11.420316,10.739667,406.142372,425.144927,4.678792,4.135869,4.329378,422.569040,4.303147,940.419327,1031.927776,9.730601,3.425062,3.758341,770.214550,824.175262,7.005933,2.805166,3.001695,2828.876711,10.302934,2713.124930,9.881360,509.377525,1.855183,17.096410,0.062266,39.960527,0.145539,...,0.000000,NaN,NaN,1.0,0.0,-100.000000,27.0,32.0,18.518519,163000.0,304000.0,86.503067,1.0,0.0,13.6,18.4,33.0,36.3,NaN,2,0,-100.000000,0.073123,0.000000,-100.000000,46.753105,46.691190,5.051535,0.389700,0.218524,0.087409,14.236807,21.906982,31433.0,32.0,1,45.2,1,0,0.0
3,1007,AL,Bibb,1044.867327,969.378841,-7.224696,4.559753,4.230324,491.449066,596.162829,21.307144,2.144661,2.601627,275.060875,224.388125,-18.422377,3.458580,2.821427,53.818772,0.676710,249.204753,232.642670,-6.645974,1.087518,1.015242,150.553323,137.688231,-8.545207,0.657008,0.600865,540.512239,2.358770,419.099942,1.828933,8.596762,0.037516,1.994318,0.008703,2.513097,0.010967,...,NaN,NaN,NaN,2.0,3.0,50.000000,5.0,6.0,20.000000,NaN,21000.0,NaN,0.0,0.0,11.1,14.8,33.0,36.3,NaN,0,1,NaN,0.000000,0.044183,NaN,75.020729,21.924504,1.771765,0.096007,0.279293,0.030548,12.681650,22.696923,40767.0,22.2,0,29.3,1,1,0.0
4,1009,AL,Blount,1548.175559,3724.428242,140.568857,2.700840,6.497380,609.027708,1650.959482,171.0811

*Part 2: Query Census API*

In [76]:
# Get Census data: county population in 2020
url = 'https://api.census.gov/data/2020/dec/pl?get=P1_001N&for=county:*'
json = requests.get(url).json()

# explore data
print(json[0])


['P1_001N', 'state', 'county']


In [77]:
# Create Census2020Pop dataframe
Census2020Pop = pd.DataFrame(columns = json[0], data = json[1:])
Census2020Pop.head()

,P1_001N,state,county
0,58805,01,001
1,231767,01,003
2,25223,01,005
3,22293,01,007
4,59134,01,009


In [78]:
# Create FIPS column to match EnvironmentAll
Census2020Pop['state'] = Census2020Pop['state'].str.lstrip('0') #strip leading 0 from state code
Census2020Pop['FIPS'] = Census2020Pop['state'] + Census2020Pop['county'] #concatenate state and county codes into single number
Census2020Pop['FIPS'] = Census2020Pop['FIPS'].astype(int) #change datatype to match EnvironmentAll['FIPS']
Census2020Pop.head()

,P1_001N,state,county,FIPS
0,58805,1,001,1001
1,231767,1,003,1003
2,25223,1,005,1005
3,22293,1,007,1007
4,59134,1,009,1009


*Part 3: Combine Datasets*

In [79]:
data = pd.merge(EnvironmentAll, Census2020Pop[['FIPS','P1_001N']], on='FIPS', how='left')

In [80]:
print(data.head())
len(data.columns)

   FIPS State   County  ...  METRO13  POPLOSS10  P1_001N
0  1001    AL  Autauga  ...        1        0.0    58805
1  1003    AL  Baldwin  ...        1        0.0   231767
2  1005    AL  Barbour  ...        0        0.0    25223
3  1007    AL     Bibb  ...        1        0.0    22293
4  1009    AL   Blount  ...        1        0.0    59134

[5 rows x 285 columns]


285